<a href="https://colab.research.google.com/github/cdmurray80/object_detection/blob/main/object_detection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Docs: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/2.2.0/install.html#tensorflow-object-detection-api-installation

In [ ]:
!nvidia-smi

In [ ]:
# Mount drive
#
#
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

In [ ]:
def do_setup():
  #
  %pip install --ignore-installed --upgrade tensorflow
  #
  
  #
  #!pip install tensorflow==2.8
  #
  #!pip install --ignore-installed --upgrade tensorflow==2.2.0
  #!pip uninstall -y tensorflow 
  #!pip install tensorflow==2.2
  !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

  import os
  if not os.path.isdir('/content/tensorflow'):
    %mkdir tensorflow
  %cd /content/tensorflow
  !git clone --depth 1 https://github.com/tensorflow/models

  %cd /content/tensorflow/models/research
  %ls
  !protoc object_detection/protos/*.proto --python_out=.

  %pip install cython
  %pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

  %cd /content/tensorflow/models/research
  !cp object_detection/packages/tf2/setup.py .
  !python -m pip install --use-feature=2020-resolver .

def test_setup():
  !python object_detection/builders/model_builder_tf2_test.py


In [ ]:
do_setup()
test_setup()

In [ ]:
import os
WORKSPACE_DIR='/content/tensorflow/workspace'
test_dir=WORKSPACE_DIR+'/training_demo/images/test'
train_dir=WORKSPACE_DIR+'/training_demo/images/train'
model_dir=WORKSPACE_DIR+'/training_demo/pre-trained-models'
annotations_dir=WORKSPACE_DIR + '/training_demo/annotations'
for d in [WORKSPACE_DIR, WORKSPACE_DIR+'/training_demo', WORKSPACE_DIR+'/training_demo/images', annotations_dir, WORKSPACE_DIR+'/training_demo/scripts', train_dir, test_dir, model_dir]:
  if not os.path.exists(d):
    os.mkdir(d)
    print('Created directory', d)

In [ ]:
#Get all xml files
import glob,shutil
import xml.etree.ElementTree as ET
paths=['/content/drive/MyDrive/Research/object_detection/images/final_MLRS', '/content/drive/MyDrive/Research/object_detection/images/final_t72']
xml_files=[glob.glob(path+'/*.xml') for path in paths]
xml_files=[f for l in xml_files for f in l]

#Move to train and test
for i,f in enumerate(xml_files):
  dir = test_dir if i % 5 == 4 else train_dir
  #Get name of actual image file
  tree=ET.parse(f)
  root=tree.getroot()
  img_file=root[1].text
  f2=f.replace('.xml', '.'+ img_file.split('.')[-1])
  shutil.copy2( f, dir)
  shutil.copy2( f2, dir)

In [ ]:
#Create label file
labels=['MLRS', 'tank']
s=''
for i,l in enumerate(labels):
  s+='''item {\n \t id: %d\n \t name:'%s'\n}\n\n''' % (i+1, l)
#Write file
label_file=annotations_dir+'/label_map.pbtxt'
text_file = open(label_file, 'w')
n = text_file.write(s)
text_file.close()

In [ ]:
# Create train and test tfrecords
train_record_file=annotations_dir+'/train.record'
test_record_file=annotations_dir+'/test.record'
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $train_dir -l $label_file -o $train_record_file
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $test_dir -l $label_file -o $test_record_file


In [ ]:
import requests

# More models are here
model_name='efficientdet_d1_coco17_tpu-32'
model_name='ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
model_name='ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'+model_name+'.tar.gz'
target_file = WORKSPACE_DIR + '/training_demo/pre-trained-models/'+model_name+'.tar.gz'
response = requests.get(url)
open(target_file, "wb").write(response.content)


In [ ]:
%cd /content/tensorflow/workspace/training_demo/pre-trained-models
!tar -xvzf $target_file 

In [ ]:
# Copy raw pipeline config to gdrive
gdrive_model_dir='/content/drive/MyDrive/Research/object_detection/'+model_name
if not os.path.exists(gdrive_model_dir):
    os.mkdir(gdrive_model_dir)
shutil.copy2(model_dir+'/' + model_name + '/pipeline.config',gdrive_model_dir+'/raw_pipeline.config')

#if I already modified it for a final version, use that
final_pipeline_cfg_file=gdrive_model_dir + '/final_pipeline.config'
assert(os.path.exists(final_pipeline_cfg_file))
shutil.copy2(final_pipeline_cfg_file, model_dir + '/' + model_name + '/pipeline.config')

#!ls /content/tensorflow/workspace/training_demo/pre-trained-models/efficientdet_d1_coco17_tpu-32
#!cp /content/tensorflow/workspace/training_demo/pre-trained-models/efficientdet_d1_coco17_tpu-32/pipeline.config /content/drive/MyDrive/Research/object_detection

In [ ]:
shutil.copy2('/content/tensorflow/models/research/object_detection/model_main_tf2.py', WORKSPACE_DIR+'/training_demo')
shutil.copy2('/content/tensorflow/models/research/object_detection/exporter_main_v2.py', WORKSPACE_DIR+'/training_demo')
%cd /content/tensorflow/workspace/training_demo

In [ ]:
%pip install "opencv-python-headless<4.3"

In [ ]:
# Actually train!
#!python model_main_tf2.py --model_dir=pre-trained-models/$model_name --pipeline_config_path=pre-trained-models/$model_name/pipeline.config
!python model_main_tf2.py --model_dir=/content/tensorflow/workspace/training_demo --pipeline_config_path=pre-trained-models/$model_name/pipeline.config
#!python model_main_tf2.py --model_dir=. --pipeline_config_path=pre-trained-models/$model_name/pipeline.config

In [ ]:
#
#
# Use model
#
#
!ls /content/tensorflow/workspace/training_demo

In [ ]:
#
# Export the model
#
%mkdir -p /content/tensorflow/workspace/training_demo/inference_graph

!python exporter_main_v2.py --trained_checkpoint_dir=/content/tensorflow/workspace/training_demo --pipeline_config_path=pre-trained-models/$model_name/pipeline.config --output_directory=/content/tensorflow/workspace/training_demo/inference_graph

In [ ]:
import glob
import random
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from object_detection.utils import ops
from object_detection.utils import visualization_utils as viz
from object_detection.utils.label_map_util import create_category_index_from_labelmap

%matplotlib inline


In [ ]:

def load_image(path):
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))
    width, height = image.size
    shape = (height, width, 3)
    image = np.array(image.getdata())
    image = image.reshape(shape).astype('uint8')
    return image

def run_inference(net, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]
    # forward pass
    model = net.signatures['serving_default']
    result = model(input_tensor)
    # extract detections
    num_detections = int(result.pop('num_detections'))
    result = {key: value[0, :num_detections].numpy() for key, value in result.items()}
    result['num_detections'] = num_detections
    result['detection_classes'] = result['detection_classes'].astype('int64')
    # use mask if available
    if 'detection_masks' in result:
        detection_masks_reframed = ops.reframe_box_masks_to_image_masks(result['detection_masks'], result['detection_boxes'], image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        result['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return result

In [ ]:
CATEGORY_IDX = create_category_index_from_labelmap(label_file, use_display_name=True)
model_path = '/content/tensorflow/workspace/training_demo/inference_graph/saved_model'
model = tf.saved_model.load(model_path)

In [ ]:
def get_image_with_boxes(model, path):
    image = load_image(path)
    annotation = run_inference(model, image)
    masks = annotation.get('detection_masks_reframed', None)
    viz.visualize_boxes_and_labels_on_image_array(
        image,
        annotation['detection_boxes'],
        annotation['detection_classes'],
        annotation['detection_scores'],
        CATEGORY_IDX,
        instance_masks=masks,
        use_normalized_coordinates=True,
        line_thickness=5)
    
    return image
    
image_paths = list(glob.glob('/content/drive/MyDrive/Research/object_detection/images/final_t72/*.jpg'))
image_paths = random.choices(image_paths, k=6)
image_paths

In [ ]:
images = [get_image_with_boxes(model, path) for path in image_paths]

In [ ]:
figure, axis = plt.subplots(2, 3, figsize=(15, 10))
for index, image in enumerate(images):
    row, col = int(index / 3), index % 3
    axis[row, col].imshow(image)
    axis[row, col].axis('off')